## **Fraud Detection in Python**

**Course Description**

A typical organization loses an estimated 5% of its yearly revenue to fraud. In this course, learn to fight fraud by using data. Apply supervised learning algorithms to detect fraudulent behavior based upon past fraud, and use unsupervised learning methods to discover new types of fraud activities. 

Fraudulent transactions are rare compared to the norm.  As such, learn to properly classify imbalanced datasets.

The course provides technical and theoretical insights and demonstrates how to implement fraud detection models. Finally, get tips and advice from real-life experience to help prevent common mistakes in fraud analytics.

**Imports**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path

**Pandas Configuration Options**

In [ ]:
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 300)
pd.set_option('display.expand_frame_repr', True)

**Data Files Location**

* Most data files for the exercises can be found on the [course site](https://www.datacamp.com/courses/fraud-detection-in-python)
    * [Chapter 1](https://assets.datacamp.com/production/repositories/2162/datasets/cc3a36b722c0806e4a7df2634e345975a0724958/chapter_1.zip)
    * [Chapter 2](https://assets.datacamp.com/production/repositories/2162/datasets/4fb6199be9b89626dcd6b36c235cbf60cf4c1631/chapter_2.zip)
    * [Chapter 3](https://assets.datacamp.com/production/repositories/2162/datasets/08cfcd4158b3a758e72e9bd077a9e44fec9f773b/chapter_3.zip)
    * [Chapter 4](https://assets.datacamp.com/production/repositories/2162/datasets/94f2356652dc9ea8f0654b5e9c29645115b6e77f/chapter_4.zip)

**Data File Objects**

In [ ]:
data = Path.cwd() / 'data' / 'fraud_detection'

ch1 = data / 'chapter_1'
cc1_file = ch1 / 'creditcard_sampledata.csv'
cc3_file = ch1 / 'creditcard_sampledata_3.csv'

ch2 = data / 'chapter_2'
cc2_file = ch2 / 'creditcard_sampledata_2.csv'

ch3 = data / 'chapter_3'
banksim_file = ch3 / 'banksim.csv'
banksim_adj_file = ch3 / 'banksim_adj.csv'
db_full_file = ch3 / 'db_full.pickle'
labels_file = ch3 / 'labels.pickle'
labels_full_file = ch3 / 'labels_full.pickle'
x_scaled_file = ch3 / 'x_scaled.pickle'
x_scaled_full_file = ch3 / 'x_scaled_full.pickle'

ch4 = data / 'chapter_4'
enron_emails_clean_file = ch4 / 'enron_emails_clean.csv'
cleantext_file = ch4 / 'cleantext.pickle'
corpus_file = ch4 / 'corpus.pickle'
dict_file = ch4 / 'dict.pickle'
ldamodel_file = ch4 / 'ldamodel.pickle'

# Introduction and preparing your data

Learn about the typical challenges associated with fraud detection. Learn how to resample data in a smart way, and tackle problems with imbalanced data.

## Introduction to fraud detection

* Types:
    * Insurance
    * Credit card
    * Identity theft
    * Money laundering
    * Tax evasion
    * Healthcare
    * Product warranty
* e-commerce businesses must continuously assess the legitimacy of client transactions
* Detecting fraud is challenging:
    * Uncommon; < 0.01% of transactions
    * Attempts are made to conceal fraud
    * Behavior evolves
    * Fraudulent activities perpetrated by networks - organized crime
* Fraud detection requires training an algorithm to identify concealed observations from any normal observations
* Fraud analytics teams:
    * Often use rules based systems, based on manually set thresholds and experience
    * Check the news
    * Receive external lists of fraudulent accounts and names
        * suspicious names or track an external hit list from police to reference check against the client base
    * Sometimes use machine learning algorithms to detect fraud or suspicious behavior
        * Existing sources can be used as inputs into the ML model
        * Verify the veracity of rules based labels

In [ ]:
df = pd.read_csv(cc1_file)
df.info()

In [ ]:
df.head()

### Checking the fraud to non-fraud ratio

### Data visualization

## Increase successful detections with data resampling

### Resampling methods for imbalanced data

### Applying Synthetic Minority Over-sampling Technique (SMOTE)

### Compare SMOTE to original data

## Fraud detection algorithms in action

### Exploring the traditional method of fraud detection

### Using ML classification to catch fraud

### Logistic regression with SMOTE

### Pipelining

# Fraud detection using labeled data

Learn how to flag fraudulent transactions with supervised learning. Use classifiers, adjust and compare them to find the most efficient fraud detection model.

## Review classification methods

### Natural hit rate

### Random Forest Classifier - part 1

### Random Forest Classifier - part 2

## Perfomance evaluation

### Performance metrics for the RF model

### Plotting the Precision vs. Recall Curve

## Adjusting the algorithm weights

### Model adjustments

### Adjusting RF for fraud detection

### Parameter optimization with GridSearchCV

### Model results with GridSearchCV

## Ensemble methods

### Logistic Regression

### Voting Classifier

### Adjusting weights within the Voting Classifier

# Fraud detection using unlabeled data

Use unsupervised learning techniques to detect fraud. Segment customers, use K-means clustering and other clustering algorithms to find suspicious occurrences in your data.

## Normal versus abnormal behavior

### Exploring the data

### Customer segmentation

### Using statistics to define normal behavior

## Clustering methods to detect fraud

### Scaling the data

### K-mean clustering

### Elbow method

## Assigning fraud vs. non-fraud

### Detecting outliers

### Checking model results

## Alternate clustering methods for fraud detection

### DB scan

### Assessing smallest clusters

### Results verification

# Fraud detection using text

Use text data, text mining and topic modeling to detect fraudulent behavior.

## Using text data

### Word search with dataframes

### Using list of terms

### Creating a flag

## Text mining to detect fraud

### Removing stopwords

### Cleaning text data

## Topic modeling on fraud

### Create dictionary and corpus

### LDA model

## Flagging fraud based on topic

### Interpreting the topic model

### Finding fraudsters based on topic

## Lesson 4: Recap